# ADS 509 Module 1: APIs and Web Scraping

This notebook has three parts. In the first part you will pull data from the Twitter API. In the second, you will scrape lyrics from AZLyrics.com. In the last part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 100,000 Twitter followers and 20 songs with lyrics on AZLyrics.com. In this part of the assignment we pull the some of the user information for the followers of your artist and store them in text files. 


## Important Note

This version of the assignment works with Tweepy version 4 (4.9 as I write this) and Version 2 of the Twitter API. Starting in Summer 2022 this should become the default version of this code. 

Run the below cell. If your version of Tweepy begins with a "4", then you should be good to go. If it begins with a "3" and you're not concerned about upgrading your package to version 4, then run the following command, found [here](https://stackoverflow.com/questions/5226311/installing-specific-package-version-with-pip), at the command line or in a cell: `pip install -Iv tweepy==4.9`. (You may want to update that version number if Tweepy has moved on past 4.9. 

In [1]:
pip show tweepy

Name: tweepy
Version: 4.10.0
Summary: Twitter library for Python
Home-page: https://www.tweepy.org/
Author: Joshua Roesslein
Author-email: tweepy@googlegroups.com
License: MIT
Location: c:\users\elfek\appdata\roaming\python\python39\site-packages
Requires: oauthlib, requests-oauthlib, requests
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Twitter API Pull

In [2]:
# for the twitter section
import tweepy
import os
import datetime
import re
from pprint import pprint

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter


In [32]:
# Use this cell for any import statements you add
import csv
import random
import shutil

We need bring in our API keys. Since API keys should be kept secret, we'll keep them in a file called `api_keys.py`. This file should be stored in the directory where you store this notebook. The example file is provided for you on Blackboard. The example has API keys that are _not_ functional, so you'll need to get Twitter credentials and replace the placeholder keys. 

In [4]:
import api_keys_v2
from api_keys_v2 import api_key, api_key_secret, bearer_token

In [5]:
client = tweepy.Client(bearer_token,wait_on_rate_limit=True)

# Testing the API

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with the handle of John Chandler, one of the instructors in this course. His handle is `@37chandler`. Feel free to use a different handle if you would like to look at someone else's data. 

We will write code to explore a few aspects of the API: 

1. Pull some of the followers @37chandler.
1. Explore response data, which gives us information about Twitter users. 
1. Pull the last few tweets by @37chandler.


In [6]:
handle = "37chandler"
user_obj = client.get_user(username=handle)

followers = client.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)



Now let's explore these a bit. We'll start by printing out names, locations, following count, and followers count for these users. 

In [7]:
num_to_print = 3

for idx, user in enumerate(followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break
    

Steve lists 'I'm over here.' as their location.
 Following: 1551, Followers: 33.

John O'Connor 🇺🇦 lists 'None' as their location.
 Following: 8, Followers: 1.

CodeGrade lists 'Amsterdam' as their location.
 Following: 2719, Followers: 390.



Let's find the person who follows this handle who has the most followers. 

In [8]:
max_followers = 0

for idx, user in enumerate(followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

        
print(max_follower_user)
print(max_follower_user.public_metrics)

JeffreyHayzlett
{'followers_count': 533085, 'following_count': 444817, 'tweet_count': 47995, 'listed_count': 5651}


Let's pull some more user fields and take a look at them. The fields can be specified in the `user_fields` argument. 

In [9]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

In [10]:
for field, value in response.data.items() :
    print(f"for {field} we have {value}")

for verified we have False
for name we have John Chandler
for public_metrics we have {'followers_count': 188, 'following_count': 573, 'tweet_count': 949, 'listed_count': 3}
for username we have 37chandler
for profile_image_url we have https://pbs.twimg.com/profile_images/2680483898/b30ae76f909352dbae5e371fb1c27454_normal.png
for location we have MN
for created_at we have 2009-04-18 22:08:22+00:00
for id we have 33029025
for description we have He/Him. Data scientist, urban cyclist, educator, erstwhile frisbee player. 

¯\_(ツ)_/¯


Now a few questions for you about the user object.

Q: How many fields are being returned in this user object? 

A: There are nine fields being returned in this user object.

---

Q: Are any of the fields within the user object non-scalar? (I.e., more complicated than a simple data type like integer, float, string, boolean, etc.) 

A: Yes, there are non-scalar fields for example the created data and the public metrics.

---

Q: How many friends, followers, and tweets does this user have? 

A: This user has 573 friends, 188 followers, and 950 tweets.


Although you won't need it for this assignment, individual tweets can be a rich source of text-based data. To illustrate the concepts, let's look at the last few tweets for this user. You are encouraged to explore the fields that are available about Tweets.

In [11]:

response = client.get_users_tweets(user_obj.data.id)

# By default, only the ID and text fields of each Tweet will be returned
for idx, tweet in enumerate(response.data) :
    print(tweet.id)
    print(tweet.text)
    print()
    
    if idx > 2 :
        break

1529510582365433856
RT @inthehands: @nycsouthpaw I asked GPT-3 to generate text based on the prompt in the first screenshot. Its output is in the second screen…

1529086669592219648
Know your trollers https://t.co/D0U43JdI6Z

1528772542982508551
RT @marksuttonbike: Quote of the month: "It's not my duty as Mayor to make sure you have a parking spot. For me it's the same as if you bou…

1528585708046360576
RT @BlackCoffeeMsla: Oh, and this weekend… https://t.co/kFWBFBwvDf



## Pulling Follower Information

In this next section of the assignment, we will pull information about the followers of your two artists. We've seen above how to pull a set of followers using `client.get_users_followers`. This function has a parameter, `max_results`, that we can use to change the number of followers that we pull. Unfortunately, we can only pull 1000 followers at a time, which means we will need to handle the _pagination_ of our results. 

The return object has the `.data` field, where the results will be found. It also has `.meta`, which we use to select the next "page" in the results using the `next_token` result. I will illustrate the ideas using our user from above. 


### Rate Limiting

Twitter limits the rates at which we can pull data, as detailed in [this guide](https://developer.twitter.com/en/docs/twitter-api/rate-limits). We can make 15 user requests per 15 minutes, meaning that we can pull $4 \cdot 15 \cdot 1000 = 60000$ users per hour. I illustrate the handling of rate limiting below, though whether or not you hit that part of the code depends on your value of `handle`.  


In the below example, I'll pull all the followers, 25 at a time. (We're using 25 to illustrate the idea; when you do this set the value to 1000.) 

TODO: Get artist counts and estimate total time.

Now let's take a look at your artists and see how long it is going to take to pull all their followers. 

In [12]:
artists = dict()
handles = ['GlassAnimals','JoynerLucas']
for handle in handles : 
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    artists[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])
    

for artist, data in artists.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")
    

It would take 2.87 hours to pull all 172412 followers for GlassAnimals. 
It would take 6.93 hours to pull all 415909 followers for JoynerLucas. 


Depending on what you see in the display above, you may want to limit how many followers you pull. It'd be great to get at least 200,000 per artist. 

As we pull data for each artist we will write their data to a folder called "twitter", so we will make that folder if needed.

In [13]:
# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

if not os.path.isdir("twitter") : 
    #shutil.rmtree("twitter/")
    os.mkdir("twitter")

In this following cells, build on the above code to pull some of the followers and their data for your two artists. As you pull the data, write the follower ids to a file called `[artist name]_followers.txt` in the "twitter" folder. For instance, for Cher I would create a file named `cher_followers.txt`. As you pull the data, also store it in an object like a list or a data frame.

Extract and store the following fields: 

* screen_name	
* name	
* id	
* location	
* followers_count	
* friends_count	
* description

 Store the fields with one user per row in a tab-delimited text file with the name `[artist name]_follower_data.txt`. For instance, for Cher I would create a file named `cher_follower_data.txt`. 


One note: the user's description can have tabs or returns in it, so make sure to clean those out of the description before writing them to the file. I've included some example code to do that below the stub. 

In [14]:
#num_followers_to_pull = 60*1000 # feel free to use this to limit the number of followers you pull.
#num_followers_to_pull = 30000
num_followers_to_pull = 200

In [15]:
handle_followers = []
pulls = 0
max_pulls = 30
next_token = None

while True :

    followers = client.get_users_followers(
        user_obj.data.id, 
        max_results=1000, # when you do this for real, set this to 1000!
        pagination_token = next_token,
        user_fields=["created_at","description","location",
                     "entities","name","pinned_tweet_id","profile_image_url",
                     "verified","public_metrics"]
    )
    pulls += 1
    
    for follower in followers.data : 
        follower_row = (follower.id,follower.name,follower.created_at,follower.description)
        handle_followers.append(follower_row)
    
    if 'next_token' in followers.meta and pulls < max_pulls :
        next_token = followers.meta['next_token']
    else : 
        break



Rate limit exceeded. Sleeping for 872 seconds.


In [60]:
# Modify the below code stub to pull the follower IDs and write them to a file. 
handle_followers = []
pulls = 0

os.chdir(r"C:\Users\elfek\datamining\twitter")

screen_name = []
name = []
ID = []
location = []
friends_count = []
followers_count = []
description = []
    
# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    output_file = handle + "_followers_data.txt"
    


    for idx, user in enumerate(followers.data) :
        screen_name.append(user.username)
        name.append(user.name)
        # Pull the follower IDs
        ID.append(user.id)
        location.append(user.location)
        followers_count.append(user.public_metrics['followers_count'])
        friends_count.append(user.public_metrics['following_count'])
        # Extract the information you need for the followers
        description.append(user.description)



    dictionaryFollowerFile = {'screen_name' : screen_name, 
                              'name' : name, 'id' : ID, 'location' : location,
                              'followers_count' : description, 'friends_count' : followers_count,
                              'description' : description}    


    # Write the data to the output file in the `twitter` folder.
    with open(output_file, 'w', encoding = 'utf8') as f:
        for i in dictionaryFollowerFile:
            f.write(str(i) + "\t")
            numSubItems = len(dictionaryFollowerFile[i])
        f.write("\n")

        for level in range(numSubItems):
            for i in dictionaryFollowerFile:
                f.write(str(dictionaryFollowerFile[i][level]) + "\t")
            f.write("\n")        
        
    # If you've pulled num_followers_to_pull, feel free to break out paged twitter API response
            

        
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)



0:00:00.016014


In [64]:
#dictFile = {'id' : ID}
handles = ['GlassAnimals','JoynerLucas']
for handle in handles :
    filepath = handle + "_followers.txt"    
    with open(filepath, 'w') as fp:
        fp.write("ID\n")
        for item in ID:
            # write each item on a new line
            fp.write("%s\n" % item)

In [18]:
tricky_description = """
    Home by Warsan Shire
    
    no one leaves home unless
    home is the mouth of a shark.
    you only run for the border
    when you see the whole city
    running as well.

"""
# This won't work in a tab-delimited text file.

clean_description = re.sub(r"\s+"," ",tricky_description)
clean_description

' Home by Warsan Shire no one leaves home unless home is the mouth of a shark. you only run for the border when you see the whole city running as well. '

---

# Lyrics Scrape

This section asks you to pull data from the Twitter API and scrape www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [28]:
artists = {'glassanimals':"https://www.azlyrics.com/g/glassanimals.html",
           'joyner':"https://www.azlyrics.com/j/joynerlucas.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: <!-- Delete this comment and put your answer here. --> 


In [29]:

# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.text, 'html.parser')
    listalbum = soup.find_all("div", "listalbum-item")
    
    links = []
    for link in soup.find_all("div", "listalbum-item"):
        text = str(link.a)
        #print(text)
        #converts ...<a href="/lyrics/joynerlucas/sugeremix.html" target="_blank"...
        # to /lyrics/joynerlucas/introskit.html
        try:
             lyricLink = re.search('href=\"(.+?)\" t', text).group(1)
        except AttributeError:
            pass
       
        if (lyricLink[0:4] != "http"):
            lyricLink = "https://www.azlyrics.com" + lyricLink
        # store the links `lyrics_pages` where the key is the artist and the
        # value is a list of links.
        lyrics_pages[artist].append(lyricLink)

Let's make sure we have enough lyrics pages to scrape. 

In [30]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [31]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For glassanimals we have 58.
The full pull will take for this artist will take 0.16 hours.
For joyner we have 109.
The full pull will take for this artist will take 0.3 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [21]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [33]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")



In [34]:
url_stub = "https://www.azlyrics.com" 
start = time.time()
total_pages = 0 

    # 1. Build a subfolder for the artist
os.chdir(r"C:\Users\elfek\datamining")
if os.path.isdir("lyrics/joyner") : 
    shutil.rmtree("lyrics/joyner/")
os.mkdir("lyrics/joyner")
os.chdir(r"C:\Users\elfek\datamining")

if os.path.isdir("lyrics/glassanimals") : 
    shutil.rmtree("lyrics/glassanimals/")
os.mkdir("lyrics/glassanimals")

for artist in lyrics_pages :

    # Use this space to carry out the following steps: 
    
    
    # 2. Iterate over the lyrics pages
    #alltext = []

    #for links in lyrics_pages[artist]:
    for links in lyrics_pages[artist]:    
    # 3. Request the lyrics page. 
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
        r = requests.get(links)
        time.sleep(5 + 10*random.random())
        # now extract the links to lyrics pages from this page
        # **Already extracted links to lyrics!!**
    
    # 4. Extract the title and lyrics from the page.
        soup = BeautifulSoup(r.text, 'html.parser')
        soup.title

        #Code to get the song title
        titlePlaceHolder = str(soup.title)
        regex = " - (.+?) Lyrics"
        try:
             songTitle = re.search(regex, titlePlaceHolder).group(1)
        except AttributeError:
            pass
        songTitle.replace("[", "")
        songTitle.replace("]", "")
        #if (songTitle != "")
        allText = ''
        for string in soup.stripped_strings:
            allText = allText + str(string) + "\n"
        #Replace non-ascii apostrophe
        allText=(re.sub('â\x80\x99', "'", allText))
        #Add ampersands instead of new lines for regex processing

        #replace non ascii characters
        #my_regex1 =  's&"' + songTitle[1:] + '"&(.+?)&Submit'
        my_regex1 =  's&"' + songTitle + '"&(.+?)&Submit'
        allLyrics = ""
        newLyrics = ""
        try:
            allLyrics = re.search(my_regex1, allText).group(1)
        except AttributeError:
            pass
        #Add back the new lines instead of the ampersands
        newLyrics = re.sub("&", "\n", allLyrics)
        #print(allLyrics)
        filename = generate_filename_from_link(links)
        os.chdir(r"C:\Users\elfek\datamining\lyrics\\" + artist)
        f = open(filename, "a", encoding="utf-8")
        f.write(songTitle + "\n\n" + newLyrics)
        f.close()
        

        
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename. 
        
            
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist

your_path = r"C:\Users\elfek\datamining\lyrics\joyner\\"
files = os.listdir(your_path)
keyword = 'your_keyword'
for file in files:
    if os.path.isfile(os.path.join(your_path, file)):
        f = open(os.path.join(your_path, file),'r')
        for x in f:
            print(x)
        f.close()
        
your_path = r"C:\Users\elfek\datamining\lyrics\glassanimals\\"
files = os.listdir(your_path)
keyword = 'your_keyword'
for file in files:
    if os.path.isfile(os.path.join(your_path, file)):
        f = open(os.path.join(your_path, file),'r')
        for x in f:
            print(x)
        f.close()
    
end = time.time()

10 Bands



ADHD



All Over



Back In Blood (Remix)



Back Words



Bank Account (Remix)



Beluxia (Skit)



Bonfire (Skit)



Broke And Stupid



Chris (Skit)



Comprehensive Evaluation (Skit)



Devil's Work



DNA. (Remix)



Don't Shoot



Dreams Unfold



Duck Duck Goose



Evolution



Fall Slowly



Father's Day (Skit)



Finally



Forever



Frozen



Fuck Your Feelings



FYM



Get In (Skit)



Gold Mine



Gucci Gang (Remix)



Half Nigga



Happy Birthday



I Don't Die



I Lied (Intro)



I Love



I'm Not Racist



Sorry



I Need More



Intro (Skit)



ISIS



Jumanji



Just Because



Just Let Go



Just Like You



Keep It 100



Kevin (Skit)



Late To The Party



Legend



Like A River



Literally



Litty Freestyle



Long Way



Look Alive (Remix)



Look Around Me



Look What You Made Me Do



Lotto



Lotto (Remix)



Lovely



Lullaby



Mansion



Mask Off Remix (Mask On)



My Escape



One Lonely Night



On This Way



Opposites Attract



Panda 

In [35]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.51 hours.


---

# Evaluation

This assignment asks you to pull data from the Twitter API and scrape www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [36]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

---

## Checking Twitter Data

The output from your Twitter API pull should be two files per artist, stored in files with formats like `cher_followers.txt` (a list of all follower IDs you pulled) and `cher_followers_data.txt`. These files should be in a folder named `twitter` within the repository directory. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [69]:
os.chdir(r"C:\Users\elfek\datamining")
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[2]}.")

We see two artist handles: GlassAnimals and JoynerLucas.


In [72]:
artist_handles = handles
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    follower_data_file = artist + "_followers_data.txt"
    
    ids = open("twitter/" + follower_file,'r', encoding = "utf8").readlines()
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    with open("twitter/" + follower_data_file,'r', encoding = "utf8") as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
        print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
        print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(words(line[6]))
            except :
                pass
    
        

        print(f"We have {idx+1} data rows for {artist} in the follower data file.")

        print(f"For {artist} we have {len(locations)} unique locations.")

        print(f"For {artist} we have {len(description_words)} words in the descriptions.")
        print("Here are the five most common words:")
        print(Counter(description_words).most_common(5))

        
        print("")
        print("-"*40)
        print("")
    

We see 2000 in your follower file for GlassAnimals, assuming a header row.
In the follower data file (GlassAnimals_followers_data.txt) for GlassAnimals, we have these columns:
screen_name : name : id : location : followers_count : friends_count : description : 

We have 1513 data rows for GlassAnimals in the follower data file.
For GlassAnimals we have 334 unique locations.
For GlassAnimals we have 4558 words in the descriptions.
Here are the five most common words:
[('i', 131), ('a', 96), ('and', 89), ('to', 78), ('the', 73)]

----------------------------------------

We see 2000 in your follower file for JoynerLucas, assuming a header row.
In the follower data file (JoynerLucas_followers_data.txt) for JoynerLucas, we have these columns:
screen_name : name : id : location : followers_count : friends_count : description : 

We have 3026 data rows for JoynerLucas in the follower data file.
For JoynerLucas we have 334 unique locations.
For JoynerLucas we have 9116 words in the descriptio

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [73]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For glassanimals we have 55 files.
For glassanimals we have roughly 151 words, 112 are unique.
For joyner we have 109 files.
For joyner we have roughly 268 words, 184 are unique.
